### 06.03. Import agents into application

In [1]:
#Reuse the agents created in the previous chapters by importing their notebooks
#Ignore / hide the code output and use only the agents

#import the Healthcare assistant agent
%run "code_02_XX A Healthcare assistant agent with ReAct.ipynb" 
print("===============================================================")
#import the appointment scheduling agent
%run "code_05_XX ReAct agent with Workflows - Doctor scheduling.ipynb"

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    
> Running step 9c7fbd9e-db58-4ed8-9a0c-da8b8f9aab14. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is: English. I need to use a tool to help me ans

### 06.04 Setup the multi-agent application

In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.tools import FunctionTool
import asyncio

#Sync agent - use as query agent directly
healthcare_agent_tool = QueryEngineTool(
    query_engine= healthcare_agent,
    metadata=ToolMetadata(
        name="Healthcare_Agent",
        description="""
            An agent that can answer questions about diseases,
            symptoms for the diseases and the medications to cure them
            """
    )
)

#Async agent - use an async function to call
async def async_doctors_and_appointments(query: str) -> str:
    response = await scheduling_agent.run(input=query)
    return response.get("response")

#Use a sync wrapper over the async function
def doctors_and_appointments(query: str) -> str :
    """
    An agent that find doctors for specific diseases & specialities
    and then schedule appointments for these doctors.

    Inputs:
    query: This is the original query from the user
    """
    response=asyncio.run(async_doctors_and_appointments(query))
    return response

appointment_scheduling_tool = FunctionTool.from_defaults(doctors_and_appointments)
    
#Create a system prompt
context = """
You are the master agent for a hospital self-service application.
You handle user requests about medications and doctors.

Use only the tools provided to handle the requests and
NOT your own memory.
"""
#Create a Multi-agent
multi_agent = ReActAgent.from_tools(
                [healthcare_agent_tool, appointment_scheduling_tool],
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


### 06.05. Execute the multi-agent application

In [3]:
response=multi_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 1cf9f7ef-61e7-457f-a325-5f8c99ef537d. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: Healthcare_Agent
Action Input: {'input': 'medication for arthritis'}
> Running step eea244f5-05bb-418f-9c3a-d7b0271a076d. Step input: medication for arthritis
Thought: The current language of the user is English. I need to use a tool to help me answer the question about medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'medication for arthritis'}
Observation: Naproxen is used for pain and inflammation, which can be associated with arthritis.
> Running step 1081ee60-a992-4706-89d5-0ace5b53c5c6. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Naproxen is a medication used for pain and inflammation, which can be associated with arthritis.
Observation: Naproxen is a medicat

In [9]:
response=multi_agent.chat("Which doctors are cardiologists?")
print("*******\n Response : ",response.response)

> Running step 2ca80bec-5848-43d9-9cae-428c755427ee. Step input: Which doctors are cardiologists?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: doctors_and_appointments
Action Input: {'query': 'cardiologists'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='The current language of the user is English. I need to use a tool to help me answer the question.' action='query_engine_tool' action_input={'input': 'cardiologists'}
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
*** Calling tools :  [ToolSelection(tool_id='fake', tool_name='query_engine_tool', tool_kwargs={'input': 'cardiologists'})]
Step handle_tool_calls produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
R

In [10]:
response=multi_agent.chat(
    "Find a neurologist and request an appointment for Trevor Hunt at the earliest")

> Running step cfe37040-e09a-40e7-afae-2e9c585db6aa. Step input: Find a neurologist and request an appointment for Trevor Hunt at the earliest
Thought: The current language of the user is English. I need to use a tool to find a neurologist and schedule an appointment for Trevor Hunt.
Action: doctors_and_appointments
Action Input: {'query': 'Find a neurologist and request an appointment for Trevor Hunt at the earliest'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='The current language of the user is English. I need to use a tool to find a neurologist.' action='query_engine_tool' action_input={'input': 'neurologist'}
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
*** Calling tools :  [ToolSelection(tool_id='fake', tool_name='query_engine_tool', tool_kwargs={'input': 'neurologist'})]
Step